In [18]:
# === 0) 환경 설정 ===
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(usecwd=True), override=True)

import os
# print("OPENAI ok:", bool(os.getenv("OPENAI_API_KEY")))  # True면 감정/브리프 OK

# === 1) 기본 임포트/스키마 ===
from typing import List, TypedDict
from pydantic import BaseModel, Field
from typing_extensions import Annotated

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

Valence = Annotated[float, Field(ge=-1.0, le=1.0)]
Arousal = Annotated[float, Field(ge=0.0, le=1.0)]
BPM     = Annotated[int,   Field(ge=50,  le=140)]
DurSec  = Annotated[int,   Field(ge=60,  le=90)]  # 60~90초

class EmotionResult(BaseModel):
    primary: str
    valence: Valence = 0.0
    arousal: Arousal = 0.5
    confidence: Arousal = 0.7
    reasons: str = "—"

class MusicBrief(BaseModel):
    mood: str
    bpm: BPM = 90
    key: str
    duration_sec: DurSec = 60
    instruments: List[str] = []
    style_tags: List[str] = []
    prompt: str  # 영어 프롬프트

class GraphState(TypedDict, total=False):
    user_text: str
    emotion: EmotionResult
    brief: MusicBrief

# === 2) LLM 헬퍼 ===
def get_llm(temp=0.2):
    return ChatOpenAI(model="gpt-4o-mini", temperature=temp)

# === 3) 감정 분석 노드 ===
def analyze_emotion_node(state: GraphState) -> GraphState:
    llm = get_llm(0.2)
    sys = (
        "당신은 심리 정서를 요약하는 분석가입니다. "
        "사용자 텍스트에서 주요 감정을 한 단어(또는 짧은 구)로 도출하고, "
        "valence(-1~1), arousal(0~1), confidence(0~1)을 추정하세요. "
        "반드시 EmotionResult(JSON 스키마)에 맞춰 응답하세요."
    )
    structured = llm.with_structured_output(EmotionResult)
    result = structured.invoke([
        {"role":"system","content":sys},
        {"role":"user","content":state["user_text"]}
    ])
    state["emotion"] = result
    return state

# === 4) 음악 브리프 노드 (생성 호출 없음) ===
def compose_brief_node(state: GraphState) -> GraphState:
    llm = get_llm(0.6)  # 브리프 다양성 조금 ↑
    emo: EmotionResult = state["emotion"]
    sys = (
    "너는 음악 감독이다. 아래 감정 분석과 사용자 텍스트를 참고해 "
    "개인의 감정과 스토리를 반영한, 예술치료/심리 안정 목적의 짧은 BGM을 위한 "
    "Music Brief를 JSON으로 만들어라.\n"
    "\n"
    "## 치료적 목표(필수)\n"
    "- 사용자의 현재 상태를 '공조절(co-regulation)' 관점에서 보정한다.\n"
    "- 정서 조절 전략(regulation_mode)을 다음에서 고른다: "
    "  soothe(불안·고각성 완화), uplift(우울·저각성 부드럽게 상승), "
    "  sustain(편안한 긍정 유지), ground(과도한 긍정/흥분을 안정적으로 접지).\n"
    "- 선택한 전략은 style_tags에 'regulate:<mode>' 형태로 반드시 포함한다.\n"
    "\n"
    "## 파라미터 규칙\n"
    "1) bpm: 50~140 중 선택하되, duration_sec은 60~90으로 제한한다.\n"
    "   - arousal↑ → bpm↑ 경향. 단, soothe/ground 전략일 때는 중간 템포(70~100)로 과자극 방지.\n"
    "   - uplift 전략(저각성·우울)일 땐 72~90 범위에서 부드럽게 추진.\n"
    "2) duration_sec: 60~90. 불안(arousal>0.6) 또는 우울(valence<-0.2)은 78~90을 우선 고려.\n"
    "3) key: valence>=0.2 → 메이저(C/G/F/D 등), valence<=-0.2 → 마이너(A/D/E/B 등), "
    "   중립은 혼합 가능. 동일 키('C major')만 반복 사용 금지.\n"
    "4) instruments: 2~4개. 기본은 warm piano, soft pad.\n"
    "   - soothe/ground: light percussion는 있어도 아주 절제(brush, soft tick 등), 하이햇/킥 과도 금지.\n"
    "   - uplift: strings(legato)나 gentle pulse로 미세한 전진감.\n"
    "5) style_tags: 3~6개. 예: calming, minimal, warm, ambient, breathing, focus, regulate:<mode>.\n"
    "6) 구조(권장): 60~90초 안에 intro(짧은 페이드인, 4bar) → body(점진적 레이어, 8~12bar) → "
    "   outro(2~4bar, 2~3초 페이드아웃). 루프 안전(loop-safe) 문장감 유지.\n"
    "7) 안전 가드: 과도한 트랜지언트/왜곡/사이드체인 펌핑/금속성 심벌/저역 과출력 금지. "
    "   다이내믹은 soft~medium.\n"
    "8) prompt: 영어 한 문장, 18~25단어. 악기·무드·질감·다이내믹을 서술하되 숫자(BPM/key/duration) 금지. "
    "   사용자의 텍스트에서 핵심 단어 1~2개를 분위기 단서로 녹여라(직역 금지, 뉘앙스만 반영).\n"
    "9) JSON만 출력. 추가 설명 금지.\n"
)
    usr = (
        f"# Emotion\nprimary={emo.primary}, valence={emo.valence}, "
        f"arousal={emo.arousal}, confidence={emo.confidence}\n\n"
        f"# Text\n{state['user_text']}\n"
    )
    structured = llm.with_structured_output(MusicBrief)
    brief = structured.invoke([
        {"role":"system","content":sys},
        {"role":"user","content":usr}
    ])

    # duration 보정 (60~90초로 강제)
    if brief.duration_sec < 60:
        brief = brief.model_copy(update={"duration_sec": 60})
    elif brief.duration_sec > 90:
        brief = brief.model_copy(update={"duration_sec": 90})

    state["brief"] = brief
    return state

# === 5) 2-노드 그래프 (생성 노드 없음 → Replicate 크레딧 0) ===
workflow = StateGraph(GraphState)
workflow.add_node("analyze_emotion", analyze_emotion_node)
workflow.add_node("compose_brief",  compose_brief_node)
workflow.add_edge(START, "analyze_emotion")
workflow.add_edge("analyze_emotion", "compose_brief")
workflow.add_edge("compose_brief", END)

graph = workflow.compile()

# === 6) 실행 예 ===
state = {
    "user_text": "차 한 잔 마시고 느긋하게 정리 모드로 들어가고 싶다."
}
final = graph.invoke(state)

print("=== Emotion ===")
print(final["emotion"].model_dump())
print("\n=== Music Brief ===")
print(final["brief"].model_dump())


=== Emotion ===
{'primary': '편안함', 'valence': 0.8, 'arousal': 0.3, 'confidence': 0.9, 'reasons': '차를 마시며 느긋하게 정리하고 싶다는 표현에서 편안한 감정이 드러남.'}

=== Music Brief ===
{'mood': 'calming', 'bpm': 80, 'key': 'C major', 'duration_sec': 75, 'instruments': ['warm piano', 'soft pad', 'light percussion'], 'style_tags': ['calming', 'ambient', 'minimal', 'regulate:sustain'], 'prompt': 'A warm piano gently flows with soft pads, creating a serene ambiance for a relaxed and organized mindset.'}
